## Overview

The "MLflow PySpark Pipeline for Diabetes Prediction" - Python Notebook is a comprehensive example of how to use the MLflow library to build a machine learning pipeline to predict diabetes in patients. The notebook demonstrates how to use various PySpark libraries, such as Pipeline, PipelineModel, Logistic Regression, BinaryClassificationEvaluator, and MulticlassClassificationEvaluator to build and evaluate the machine learning model.

The first step is to set the MLflow experiment for diabetes prediction. The next step is to load the diabetes dataset, which is split into training and test sets using the randomSplit function. The hyperparameters for the logistic regression model are set, and the model is trained using the pipeline function.

The categorical features in the dataset are converted to numeric using StringIndexer, and the features vector is created using VectorAssembler. The logistic regression model is defined, and the pipeline is created using these components. The hyperparameters are logged, and the model is trained.

After training the model, the predictions are made on the test data, and the model is evaluated using BinaryClassificationEvaluator and MulticlassClassificationEvaluator. The evaluation metrics, such as area under the ROC curve and accuracy, are logged using the mlflow.log_metric function.

The model is then logged using the mlflow.spark.log_model function, which saves the model to the MLflow registry. Finally, the production version of the model is transitioned using the MlflowClient function. Then, The model is then loaded and used to make predictions on new data.

This notebook is written in **Python** so the default cell type is Python.

In [ ]:
import mlflow
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

# Set the MLflow experiment
mlflow.set_experiment("/diabetes-prediction")

# Load the diabetes data
data = spark.read.csv("/FileStore/tables/diabetes_prediction_dataset.csv", inferSchema=True,header=True)

# Convert the diabetes label to double
data = data.withColumn("diabetes", col("diabetes").cast(DoubleType()))

# Split the data into training and test sets
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=1234)

# Set the hyperparameters for the logistic regression model
lr_params = {
    "regParam": 0.01,
    "elasticNetParam": 0.0,
    "maxIter": 100
}


# Train the model and make predictions on the test data
with mlflow.start_run():

    # Convert categorical features to numeric using StringIndexer
    GenderIndexer = StringIndexer(inputCol="gender", outputCol="genderIndex")                       
    SmokeHistIndexer = StringIndexer(inputCol="smoking_history", outputCol="smoking_statusIndex")  

    # Create the features vector using VectorAssembler
    F_assembler = VectorAssembler(inputCols=["genderIndex", "age", "hypertension", "heart_disease", "smoking_statusIndex", "bmi","HbA1c_level","blood_glucose_level"], outputCol="features")
    
    # Define the logistic regression model
    lr = LogisticRegression(featuresCol="features", labelCol="diabetes")
  
    # Define the pipeline
    pipeline = Pipeline(stages=[GenderIndexer, SmokeHistIndexer,F_assembler, lr])
    
    # Log the hyperparameters
    mlflow.log_params(lr_params)
    
    # Train the model
    models=pipeline.fit(trainingData)
    
    # Make predictions on the test data
    predictions = models.transform(testData)
    
    # Evaluate the model
    auc_evaluator = BinaryClassificationEvaluator(labelCol="diabetes", rawPredictionCol="prediction", metricName="areaUnderROC")
    areaUnderROC = auc_evaluator.evaluate(predictions)
    print("Area under ROC curve: {}".format(areaUnderROC))
    acc_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="diabetes", metricName="accuracy")
    accuracy = acc_evaluator.evaluate(predictions)
    print("Accuracy: {:.2f}%".format(accuracy*100))
    
    #Log the metrics
    mlflow.log_metric("Accuracy", accuracy*100)
    mlflow.log_metric("AUC", areaUnderROC)
    
    # Log the model
    mlflow.spark.log_model(models,"model")

    #Get the ID of the current run
    runid=mlflow.active_run().info.run_id

Area under ROC curve: 0.8106047708876916
Accuracy: 96.11%


2023/04/07 15:56:51 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
/databricks/python/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
runid #runid contains the unique identifier of the active MLflow run
model_uri='runs:/'+runid+'/model' #Here we are constructing the model URI using the run ID of the active run and the 'model' endpoint
model_name="Diabetes_prediction_model" #Set the name for the registered model
model_details=mlflow.register_model(model_uri=model_uri,name=model_name) #Register the trained model with the name and URI specified

Registered model 'Diabetes_prediction_model' already exists. Creating a new version of this model...
2023/04/07 15:57:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Diabetes_prediction_model, version 1
Created version '1' of model 'Diabetes_prediction_model'.


In [ ]:
from mlflow.tracking.client import MlflowClient  #Import the Mlflow client library

client=MlflowClient() #Create a client instance for accessing the Mlflow tracking server
client.transition_model_version_stage(name=model_details.name,version=model_details.version,stage='Production') #Staging the registered model version to the production stage

Out[4]: <ModelVersion: creation_timestamp=1680883044693, current_stage='Production', description='', last_updated_timestamp=1680883054203, name='Diabetes_prediction_model', run_id='49de1c4521b14bb09ae33697c3a38aaf', run_link='', source='dbfs:/databricks/mlflow-tracking/4430474130799451/49de1c4521b14bb09ae33697c3a38aaf/artifacts/model', status='READY', status_message='', tags={}, user_id='3184715775520627', version='1'>

In [ ]:
import mlflow

#Load the saved MLflow model
loaded_model = mlflow.spark.load_model(model_uri)

#Create a new test dataset as a Spark dataframe
test = spark.createDataFrame([
    ('Female',39.0,1,0,"No Info",79,8.8,145)], ["gender", "age", "hypertension","heart_disease","smoking_history","bmi","HbA1c_level","blood_glucose_level"])

#Use the loaded model to make predictions on the new dataset
new_predictions=loaded_model.transform(test)
#Display the predicted output
new_predictions.show()

2023/04/07 15:57:34 INFO mlflow.spark: 'runs:/49de1c4521b14bb09ae33697c3a38aaf/model' resolved as 'dbfs:/databricks/mlflow-tracking/4430474130799451/49de1c4521b14bb09ae33697c3a38aaf/artifacts/model'


+------+----+------------+-------------+---------------+---+-----------+-------------------+-----------+-------------------+--------------------+--------------------+--------------------+----------+
|gender| age|hypertension|heart_disease|smoking_history|bmi|HbA1c_level|blood_glucose_level|genderIndex|smoking_statusIndex|            features|       rawPrediction|         probability|prediction|
+------+----+------------+-------------+---------------+---+-----------+-------------------+-----------+-------------------+--------------------+--------------------+--------------------+----------+
|Female|39.0|           1|            0|        No Info| 79|        8.8|                145|        0.0|                0.0|[0.0,39.0,1.0,0.0...|[-7.4964843174006...|[5.54724371398871...|       1.0|
+------+----+------------+-------------+---------------+---+-----------+-------------------+-----------+-------------------+--------------------+--------------------+--------------------+----------+

